# Noise on spacecrafts

Before, we'll start. The imports and some preparation.

In [16]:
from gravann.util import get_target_point_sampler
from gravann.labels import binder as label_binder
from gravann.functions import unit_conversion as unit_converter

import numpy as np
import torch

# churyumov-gerasimenko, eros, bennu ...
SAMPLE = "churyumov-gerasimenko"

acc_fn = label_binder.bind_label("polyhedral", sample=SAMPLE)

This notebook explores the effects of solar radiation pressure (SRP) on the acceleration of a spacecraft. Further, it calculates a suitable Gaussian noise representing distributions of an accelerometers.

## Gaussian Noise - Gradiometer Inprecision

Let's first use inaccuracy. The [GOCE](https://en.wikipedia.org/wiki/Gravity_Field_and_Steady-State_Ocean_Circulation_Explorer#cite_note-home-7) mission used a "highly sensitive gravity gradiometer consisting of three pairs of accelerometers which measured gravitational gradients along three orthogonal axes." It was designed to determine gravity-field anomalies with a precision of up to
$$
10^{-5} \frac{m}{s^2} = 1 mGal
$$

If we have a look at our two examples Eros and Churyumov–Gerasimenko, we'll have the following conversion factors and acceleration magnitudes.

Let's start with some reasonable altitudes (from the surface of course):

In [17]:
altitudes_norm = np.array([0.1, 0.5, 1.0, 5.0])
altitudes_real = unit_converter.convert_altitude(SAMPLE, altitudes_norm) / 1000.0

points = []
for alt in altitudes_norm:
    target_sampler = get_target_point_sampler(
        N=1, method="altitude", bounds=[alt],
        limit_shape_to_asteroid=f"./3dmeshes/{SAMPLE}.pk", seed=42
    )
    p = target_sampler()
    points.append(p)
points = torch.concatenate(points)
print(points)

tensor([[-0.5028,  0.6385,  0.1428],
        [-0.3490,  1.2492,  0.2721],
        [-0.7257,  1.6392,  0.3664],
        [-0.4965,  1.5836, -5.2992]], dtype=torch.float64)


Now, we calculate the acceleration at these points:

In [18]:
acc_norm = acc_fn(points)
acc_real = unit_converter.convert_acceleration(SAMPLE, acc_norm)

In [19]:
print(f"-----------NORMALIZED-----------")
print(f"The acceleration at normalized altitudes {altitudes_norm}:")
print(acc_norm)
print(f"-----------REAL-----------")
print(f"The acceleration at real altitudes {altitudes_real} [km] in [m/s^2]:")
print(acc_real)

-----------NORMALIZED-----------
The acceleration at normalized altitudes [0.1 0.5 1.  5. ]:
tensor([[ 1.5940, -1.0660, -0.5360],
        [ 0.1861, -0.6291, -0.1557],
        [ 0.1342, -0.2840, -0.0687],
        [ 0.0029, -0.0091,  0.0309]])
-----------REAL-----------
The acceleration at real altitudes [ 0.31266064  1.56330322  3.12660645 15.63303223] [km] in [m/s^2]:
tensor([[ 1.0863e-04, -7.2650e-05, -3.6532e-05],
        [ 1.2682e-05, -4.2875e-05, -1.0610e-05],
        [ 9.1477e-06, -1.9356e-05, -4.6852e-06],
        [ 1.9680e-07, -6.2271e-07,  2.1026e-06]])


As we can see, the acceleration from 1.5 kilometers onward are the in the order of magnitude of the gradiometer's precision. So assuming a certain deviation is reasonable. We model this by adding gaussian noise to the calculated acceleration label for training simulating in-precisions.

But what are reasonable Gaussain parameters?

If we have a look at the values above we can simply "calculate backwards".

In [20]:
gaussian_std_real = torch.tensor(10e-5)
gaussian_std_normal = unit_converter.convert_acceleration(SAMPLE, gaussian_std_real, False)
print(f"Standard deviation for training: {gaussian_std_normal}")

Standard deviation for training: 1.4673142433166504


The experiments, we already did, had the following values:
$$
\mu = 0 \\
\sigma = 1.0
$$

The above calculation shows that this was already not that bad, but more noise is reasonable!

## Constant Bias - Solar Radiation Pressure

This probably a bit simplified, but assuming our spacecraft is always facing in the same way to the sun with the maximal cross-section and the asteroid does not reflect a lot. If so, we assume that just one cartesian vector component is affected by the Bias. *Inner Pig-Dog, you sure?*

$$
P = \frac{S_0}{c} \cdot (\frac{1}{AU}) ^2
$$
where $S_0$, $c$, $AU$ are a solar constant, a speed of light, and a distance between the sun and the solar sail expressed in an astronomical units.